In [1]:
pip install pandas==1.5.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 84.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [2]:
import pandas as pd
pd.__version__

'1.5.3'

# 아파트 거래 수정1(변화율 컬럼 추가)

In [ ]:
# 아파트 거래 테이블 불러오기기
final_df2 = pd.read_pickle('/content/drive/MyDrive/house_price/final_df2.pkl')
final_df2.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price
0,2011,1,5,강남구,개포동,138,0,89400,11000,778
1,2011,1,5,강남구,개포동,185,0,89000,22000,1786
2,2011,1,6,강남구,개포동,138,0,89400,7500,778
3,2011,1,6,강남구,개포동,185,0,89000,30666,1424
4,2011,1,7,강남구,개포동,12,0,35000,24000,1196


In [ ]:
final_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24144106 entries, 0 to 24144105
Data columns (total 10 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   year             int16 
 1   month            int16 
 2   day              int16 
 3   address_1        object
 4   address_2        object
 5   address_3        int16 
 6   address_4        int16 
 7   deal_price       int32 
 8   full_rent_price  int32 
 9   year_rent_price  int32 
dtypes: int16(5), int32(3), object(2)
memory usage: 875.0+ MB


## 해당달의 평균 매매,전세,월세 가격들은 과거달의 평균들과 비교 

- 거래개수, 지표등은 해당달의 수치를 다음달에 알 수 있지만 체결가격들은 굳이 한달 씩 미룰 필요는 없을 듯

In [ ]:
# 지역별 달별 거래 평균을 구함함
group_final_df = final_df2.groupby(['address_1','address_2','address_3','address_4','year','month']).agg({'deal_price': 'mean','full_rent_price':'mean','year_rent_price':'mean'}).copy()
group_final_df.reset_index(inplace=True)
group_final_df.head(10)

,address_1,address_2,address_3,address_4,year,month,deal_price,full_rent_price,year_rent_price
0,강남구,개포동,12,0,2011,1,40938.000000,18332.640000,1190.480000
1,강남구,개포동,12,0,2011,2,46330.357143,16232.107143,1176.464286
2,강남구,개포동,12,0,2011,3,47822.580645,17971.774194,1256.774194
3,강남구,개포동,12,0,2011,4,45638.666667,16180.866667,1026.800000
4,강남구,개포동,12,0,2011,5,41406.451613,18709.645161,1200.387097
5,강남구,개포동,12,0,2011,6,42056.666667,19296.066667,1096.966667
6,강남구,개포동,12,0,2011,7,42387.096774,20866.129032,1332.838710
7,강남구,개포동,12,0,2011,8,44316.129032,20264.516129,1228.387097
8,강남구,개포동,12,0,2011,9,43208.333333,20738.866667,1133.933333
9,강남구,개포동,12,0,2011,10,35287.096774,21646.451613,1315.354839


In [ ]:
import numpy as np
# 1개월 전 평균 가격들을 계산
group_final_df['deal_price_mean_1m_before'] = group_final_df['deal_price']
group_final_df['full_rent_price_mean_1m_before'] = group_final_df['full_rent_price']
group_final_df['year_rent_price_mean_1m_before'] = group_final_df['year_rent_price']
# 밀리면서 사라질 값들을 null 값 설정, null로 안하면 다른 지역값에 더어쓰기가 됨
group_final_df.loc[(group_final_df['year']==2022)&(group_final_df['month']==12),
                   ['deal_price_mean_1m_before', 'full_rent_price_mean_1m_before',
                    'year_rent_price_mean_1m_before']] = np.nan
# 한 칸씩 미뤄서 이전달의 결과를 얻음음
group_final_df[['deal_price_mean_1m_before', 'full_rent_price_mean_1m_before',
                'year_rent_price_mean_1m_before']] = group_final_df[['deal_price_mean_1m_before', 
                                                                     'full_rent_price_mean_1m_before',
                                                                     'year_rent_price_mean_1m_before']].shift(1)

# 3개월 전 평균 가격들을 계산
group_final_df['deal_price_mean_3m_before'] = group_final_df['deal_price']
group_final_df['full_rent_price_mean_3m_before'] = group_final_df['full_rent_price']
group_final_df['year_rent_price_mean_3m_before'] = group_final_df['year_rent_price']
for i in range(3):
    number=12-i
    group_final_df.loc[(group_final_df['year']==2022)&(group_final_df['month']==number),
                        ['deal_price_mean_3m_before', 'full_rent_price_mean_3m_before',
                         'year_rent_price_mean_3m_before']]=np.nan
group_final_df[['deal_price_mean_3m_before', 'full_rent_price_mean_3m_before',
                 'year_rent_price_mean_3m_before']] = group_final_df[['deal_price_mean_3m_before',
                 'full_rent_price_mean_3m_before','year_rent_price_mean_3m_before']].shift(3)

# 6개월 전 평균 가격들을 계산
group_final_df['deal_price_mean_6m_before'] = group_final_df['deal_price']
group_final_df['full_rent_price_mean_6m_before'] = group_final_df['full_rent_price']
group_final_df['year_rent_price_mean_6m_before'] = group_final_df['year_rent_price']
for i in range(6):
    number=12-i
    group_final_df.loc[(group_final_df['year']==2022)&(group_final_df['month']==number),
                        ['deal_price_mean_6m_before', 'full_rent_price_mean_6m_before',
                         'year_rent_price_mean_6m_before']]=np.nan
group_final_df[['deal_price_mean_6m_before', 'full_rent_price_mean_6m_before',
                 'year_rent_price_mean_6m_before']] = group_final_df[['deal_price_mean_6m_before',
                 'full_rent_price_mean_6m_before','year_rent_price_mean_6m_before']].shift(6)

# 12개월 전 평균 가격들을 계산
group_final_df['deal_price_mean_12m_before'] = group_final_df['deal_price']
group_final_df['full_rent_price_mean_12m_before'] = group_final_df['full_rent_price']
group_final_df['year_rent_price_mean_12m_before'] = group_final_df['year_rent_price']
group_final_df.loc[(group_final_df['year']==2022),
                        ['deal_price_mean_12m_before', 'full_rent_price_mean_12m_before',
                         'year_rent_price_mean_12m_before']]=np.nan
group_final_df[['deal_price_mean_12m_before', 'full_rent_price_mean_12m_before',
                 'year_rent_price_mean_12m_before']] = group_final_df[['deal_price_mean_12m_before',
                 'full_rent_price_mean_12m_before','year_rent_price_mean_12m_before']].shift(12)

group_final_df.drop(['deal_price','full_rent_price','year_rent_price'],axis=1, inplace=True)
group_final_df.dropna(inplace=True)
group_final_df.head(10)      

,address_1,address_2,address_3,address_4,year,month,deal_price_mean_1m_before,full_rent_price_mean_1m_before,year_rent_price_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before
12,강남구,개포동,12,0,2012,1,42426.709677,21678.483871,1155.290323,35287.096774,21646.451613,1315.354839,42387.096774,20866.129032,1332.838710,40938.000000,18332.640000,1190.480000
13,강남구,개포동,12,0,2012,2,30774.193548,19862.774194,1112.387097,53386.666667,21059.133333,1280.266667,44316.129032,20264.516129,1228.387097,46330.357143,16232.107143,1176.464286
14,강남구,개포동,12,0,2012,3,40358.620690,20747.275862,1088.896552,42426.709677,21678.483871,1155.290323,43208.333333,20738.866667,1133.933333,47822.580645,17971.774194,1256.774194
15,강남구,개포동,12,0,2012,4,43870.967742,19877.935484,1057.032258,30774.193548,19862.774194,1112.387097,35287.096774,21646.451613,1315.354839,45638.666667,16180.866667,1026.800000
16,강남구,개포동,12,0,2012,5,44583.333333,19611.533333,1185.033333,40358.620690,20747.275862,1088.896552,53386.666667,21059.133333,1280.266667,41406.451613,18709.645161,1200.387097
17,강남구,개포동,12,0,2012,6,42716.129032,19727.129032,1055.677419,43870.967742,19877.935484,1057.032258,42426.709677,21678.483871,1155.290323,42056.666667,19296.066667,1096.966667
18,강남구,개포동,12,0,2012,7,43401.666667,19952.733333,1163.833333,44583.333333,19611.533333,1185.033333,30774.193548,19862.774194,1112.387097,42387.096774,20866.129032,1332.838710
19,강남구,개포동,12,0,2012,8,32402.419355,18897.806452,1070.645161,42716.129032,19727.129032,1055.677419,40358.620690,20747.275862,1088.896552,44316.129032,20264.516129,1228.387097
20,강남구,개포동,12,0,2012,9,35403.225806,19605.354839,1282.000000,43401.666667,19952.733333,1163.833333,43870.967742,19877.935484,1057.032258,43208.333333,20738.866667,1133.933333
21,강남구,개포동,12,0,2012,10,32816.666667,22546.366667,1250.200000,32402.419355,18897.806452,1070.645161,44583.333333,19611.533333,1185.033333,35287.096774,21646.451613,1315.354839


In [ ]:
group_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707038 entries, 12 to 796746
Data columns (total 18 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   address_1                        707038 non-null  object 
 1   address_2                        707038 non-null  object 
 2   address_3                        707038 non-null  int64  
 3   address_4                        707038 non-null  int64  
 4   year                             707038 non-null  int64  
 5   month                            707038 non-null  int64  
 6   deal_price_mean_1m_before        707038 non-null  float64
 7   full_rent_price_mean_1m_before   707038 non-null  float64
 8   year_rent_price_mean_1m_before   707038 non-null  float64
 9   deal_price_mean_3m_before        707038 non-null  float64
 10  full_rent_price_mean_3m_before   707038 non-null  float64
 11  year_rent_price_mean_3m_before   707038 non-null  float64
 12  d

In [ ]:
group_final_df = group_final_df.astype({'address_3':'int16','address_4':'int16','year':'int16','month':'int16',
                       'deal_price_mean_1m_before':'int32', 'full_rent_price_mean_1m_before':'int32', 'year_rent_price_mean_1m_before':'int32',
                       'deal_price_mean_3m_before':'int32', 'full_rent_price_mean_3m_before':'int32', 'year_rent_price_mean_3m_before':'int32',
                       'deal_price_mean_6m_before':'int32', 'full_rent_price_mean_6m_before':'int32', 'year_rent_price_mean_6m_before':'int32',
                       'deal_price_mean_12m_before':'int32', 'full_rent_price_mean_12m_before':'int32', 'year_rent_price_mean_12m_before':'int32'})

In [ ]:
group_final_df.head()

,address_1,address_2,address_3,address_4,year,month,deal_price_mean_1m_before,full_rent_price_mean_1m_before,year_rent_price_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before
12,강남구,개포동,12,0,2012,1,42426,21678,1155,35287,21646,1315,42387,20866,1332,40938,18332,1190
13,강남구,개포동,12,0,2012,2,30774,19862,1112,53386,21059,1280,44316,20264,1228,46330,16232,1176
14,강남구,개포동,12,0,2012,3,40358,20747,1088,42426,21678,1155,43208,20738,1133,47822,17971,1256
15,강남구,개포동,12,0,2012,4,43870,19877,1057,30774,19862,1112,35287,21646,1315,45638,16180,1026
16,강남구,개포동,12,0,2012,5,44583,19611,1185,40358,20747,1088,53386,21059,1280,41406,18709,1200


In [ ]:
group_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707038 entries, 12 to 796746
Data columns (total 18 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   address_1                        707038 non-null  object
 1   address_2                        707038 non-null  object
 2   address_3                        707038 non-null  int16 
 3   address_4                        707038 non-null  int16 
 4   year                             707038 non-null  int16 
 5   month                            707038 non-null  int16 
 6   deal_price_mean_1m_before        707038 non-null  int32 
 7   full_rent_price_mean_1m_before   707038 non-null  int32 
 8   year_rent_price_mean_1m_before   707038 non-null  int32 
 9   deal_price_mean_3m_before        707038 non-null  int32 
 10  full_rent_price_mean_3m_before   707038 non-null  int32 
 11  year_rent_price_mean_3m_before   707038 non-null  int32 
 12  deal_price_mean

In [ ]:
final_df3 = pd.merge(final_df2, group_final_df, 
         on=['address_1','address_2','address_3','address_4','year','month'], how='left')

In [ ]:
final_df3.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,year_rent_price_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before
0,2011,1,5,강남구,개포동,138,0,89400,11000,778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,1,5,강남구,개포동,185,0,89000,22000,1786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,1,6,강남구,개포동,138,0,89400,7500,778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,1,6,강남구,개포동,185,0,89000,30666,1424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,1,7,강남구,개포동,12,0,35000,24000,1196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
final_df3.isnull().sum() # 2011년 자료들이 null 값들이 됨(12개월 전 평균 가격들 때문에)

year                                     0
month                                    0
day                                      0
address_1                                0
address_2                                0
address_3                                0
address_4                                0
deal_price                               0
full_rent_price                          0
year_rent_price                          0
deal_price_mean_1m_before          2620275
full_rent_price_mean_1m_before     2620275
year_rent_price_mean_1m_before     2620275
deal_price_mean_3m_before          2620275
full_rent_price_mean_3m_before     2620275
year_rent_price_mean_3m_before     2620275
deal_price_mean_6m_before          2620275
full_rent_price_mean_6m_before     2620275
year_rent_price_mean_6m_before     2620275
deal_price_mean_12m_before         2620275
full_rent_price_mean_12m_before    2620275
year_rent_price_mean_12m_before    2620275
dtype: int64

In [ ]:
# null 있는 row들 제거거
final_df3.dropna(inplace=True)

In [ ]:
# 변화율 = 100*((현재가격-과거가격)/과거가격)
final_df3['deal_price_mean_1m_before'] = 100*((final_df3['deal_price']-final_df3['deal_price_mean_1m_before'])/final_df3['deal_price_mean_1m_before'])
final_df3['full_rent_price_mean_1m_before'] = 100*((final_df3['full_rent_price']-final_df3['full_rent_price_mean_1m_before'])/final_df3['full_rent_price_mean_1m_before'])
final_df3['year_rent_price_mean_1m_before'] = 100*((final_df3['year_rent_price']-final_df3['year_rent_price_mean_1m_before'])/final_df3['year_rent_price_mean_1m_before'])

final_df3['deal_price_mean_3m_before'] = 100*((final_df3['deal_price']-final_df3['deal_price_mean_3m_before'])/final_df3['deal_price_mean_3m_before'])
final_df3['full_rent_price_mean_3m_before'] = 100*((final_df3['full_rent_price']-final_df3['full_rent_price_mean_3m_before'])/final_df3['full_rent_price_mean_3m_before'])
final_df3['year_rent_price_mean_3m_before'] = 100*((final_df3['year_rent_price']-final_df3['year_rent_price_mean_3m_before'])/final_df3['year_rent_price_mean_3m_before'])

final_df3['deal_price_mean_6m_before'] = 100*((final_df3['deal_price']-final_df3['deal_price_mean_6m_before'])/final_df3['deal_price_mean_6m_before'])
final_df3['full_rent_price_mean_6m_before'] = 100*((final_df3['full_rent_price']-final_df3['full_rent_price_mean_6m_before'])/final_df3['full_rent_price_mean_6m_before'])
final_df3['year_rent_price_mean_6m_before'] = 100*((final_df3['year_rent_price']-final_df3['year_rent_price_mean_6m_before'])/final_df3['year_rent_price_mean_6m_before'])

final_df3['deal_price_mean_12m_before'] = 100*((final_df3['deal_price']-final_df3['deal_price_mean_12m_before'])/final_df3['deal_price_mean_12m_before'])
final_df3['full_rent_price_mean_12m_before'] = 100*((final_df3['full_rent_price']-final_df3['full_rent_price_mean_12m_before'])/final_df3['full_rent_price_mean_12m_before'])
final_df3['year_rent_price_mean_12m_before'] = 100*((final_df3['year_rent_price']-final_df3['year_rent_price_mean_12m_before'])/final_df3['year_rent_price_mean_12m_before'])


In [ ]:
final_df3.head(10)

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,year_rent_price_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before
4847,2012,1,1,강남구,개포동,12,0,26500,15000,1172,...,1.471861,-24.901522,-30.703132,-10.874525,-37.480831,-28.112719,-12.012012,-35.267966,-18.175867,-1.512605
4849,2012,1,1,강남구,개포동,138,0,60000,12000,665,...,-11.920530,-3.074164,46.092038,8.660131,-10.886678,48.957299,10.465116,-24.552028,26.275913,-7.638889
4850,2012,1,1,강남구,개포동,140,0,98800,25000,1170,...,-2.743142,42.490409,45.070504,33.409350,67.307334,39.828849,6.074343,9.741197,64.171263,-4.723127
4851,2012,1,1,강남구,개포동,141,0,96500,11500,731,...,-12.244898,23.071037,2.376925,-13.079667,15.409914,19.072272,-0.136612,32.345882,9.806168,-9.079602
4854,2012,1,1,강남구,개포동,185,0,80800,35500,1644,...,1.669759,4.533223,5.188302,-19.608802,7.687387,10.958305,-8.310095,-1.844068,31.696097,-1.261261
4855,2012,1,1,강남구,개포동,187,0,61000,22000,2390,...,18.199802,-26.469702,-32.773109,38.470452,-19.152828,-32.215923,58.383035,31.343798,-26.095136,25.327740
4856,2012,1,1,강남구,개포동,189,0,63000,10000,680,...,-8.232119,-4.109589,4.036621,-2.439024,-7.920315,0.908174,2.255639,51.602657,11.209964,-5.424200
4857,2012,1,1,강남구,개포동,649,0,120800,50000,3430,...,0.000000,25.496063,27.625903,2.265951,-3.234592,-9.974793,9.760000,21.407035,3.092784,56.621005
4862,2012,1,2,강남구,개포동,12,0,26500,23500,1172,...,1.471861,-24.901522,8.565093,-10.874525,-37.480831,12.623406,-12.012012,-35.267966,28.191141,-1.512605
4864,2012,1,2,강남구,개포동,138,0,60000,12000,665,...,-11.920530,-3.074164,46.092038,8.660131,-10.886678,48.957299,10.465116,-24.552028,26.275913,-7.638889


In [ ]:
final_df3.tail(10)

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,year_rent_price_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before
24144096,2022,12,31,중랑구,중화동,307,6,43900,30000,1234,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.603622,0.000000,0.000000,-22.344171,0.000000
24144097,2022,12,31,중랑구,중화동,307,76,37800,33600,1304,...,0.000000,0.000000,0.000000,0.000000,0.000000,11.258278,0.000000,0.000000,11.258278,0.000000
24144098,2022,12,31,중랑구,중화동,314,1,74000,37000,2244,...,54.332875,0.000000,0.000000,1.538462,0.000000,60.967545,1.081081,4.225352,-21.276596,26.923077
24144099,2022,12,31,중랑구,중화동,318,81,32500,12000,298,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24144100,2022,12,31,중랑구,중화동,331,64,76500,15500,1000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24144101,2022,12,31,중랑구,중화동,438,0,69500,32000,389,...,0.000000,5.677706,0.000000,0.000000,11.200000,17.216117,0.000000,11.200000,39.130435,0.000000
24144102,2022,12,31,중랑구,중화동,450,0,91500,32500,2090,...,2.955665,0.000000,-5.531494,-1.415094,25.342466,0.135568,17.021277,9.975962,1.217727,8.066184
24144103,2022,12,31,중랑구,중화동,452,0,55000,33000,1610,...,0.000000,0.000000,-26.666667,8.783784,0.000000,-26.666667,8.783784,0.000000,-26.666667,8.783784
24144104,2022,12,31,중랑구,중화동,453,0,85500,57750,2070,...,0.000000,0.000000,-0.431034,0.000000,0.000000,15.500000,0.000000,6.875000,15.500000,3.967855
24144105,2022,12,31,중랑구,중화동,454,0,96000,40000,2140,...,0.000000,0.000000,-4.761905,0.000000,0.000000,-4.761905,0.000000,0.608893,-11.406423,0.000000


In [ ]:
final_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21523831 entries, 4847 to 24144105
Data columns (total 22 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   year                             int16  
 1   month                            int16  
 2   day                              int16  
 3   address_1                        object 
 4   address_2                        object 
 5   address_3                        int16  
 6   address_4                        int16  
 7   deal_price                       int32  
 8   full_rent_price                  int32  
 9   year_rent_price                  int32  
 10  deal_price_mean_1m_before        float64
 11  full_rent_price_mean_1m_before   float64
 12  year_rent_price_mean_1m_before   float64
 13  deal_price_mean_3m_before        float64
 14  full_rent_price_mean_3m_before   float64
 15  year_rent_price_mean_3m_before   float64
 16  deal_price_mean_6m_before        float64
 17  ful

In [ ]:
# float64 타입 변경경
final_df3 = final_df3.astype({'deal_price_mean_1m_before':'float32', 'full_rent_price_mean_1m_before':'float32', 'year_rent_price_mean_1m_before':'float32',
                              'deal_price_mean_3m_before':'float32', 'full_rent_price_mean_3m_before':'float32', 'year_rent_price_mean_3m_before':'float32',
                              'deal_price_mean_6m_before':'float32', 'full_rent_price_mean_6m_before':'float32', 'year_rent_price_mean_6m_before':'float32',
                              'deal_price_mean_12m_before':'float32', 'full_rent_price_mean_12m_before':'float32', 'year_rent_price_mean_12m_before':'float32'})

In [ ]:
final_df3.head(10)

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,year_rent_price_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before
4847,2012,1,1,강남구,개포동,12,0,26500,15000,1172,...,1.471861,-24.901522,-30.703133,-10.874525,-37.480831,-28.112719,-12.012012,-35.267967,-18.175867,-1.512605
4849,2012,1,1,강남구,개포동,138,0,60000,12000,665,...,-11.920529,-3.074164,46.092037,8.660131,-10.886678,48.957298,10.465117,-24.552027,26.275913,-7.638889
4850,2012,1,1,강남구,개포동,140,0,98800,25000,1170,...,-2.743142,42.490410,45.070503,33.409351,67.307335,39.828850,6.074343,9.741198,64.171265,-4.723127
4851,2012,1,1,강남구,개포동,141,0,96500,11500,731,...,-12.244898,23.071037,2.376925,-13.079667,15.409914,19.072271,-0.136612,32.345882,9.806169,-9.079602
4854,2012,1,1,강남구,개포동,185,0,80800,35500,1644,...,1.669759,4.533223,5.188302,-19.608803,7.687387,10.958304,-8.310095,-1.844068,31.696096,-1.261261
4855,2012,1,1,강남구,개포동,187,0,61000,22000,2390,...,18.199802,-26.469702,-32.773109,38.470451,-19.152828,-32.215923,58.383034,31.343798,-26.095137,25.327740
4856,2012,1,1,강남구,개포동,189,0,63000,10000,680,...,-8.232119,-4.109589,4.036621,-2.439024,-7.920314,0.908174,2.255639,51.602657,11.209965,-5.424200
4857,2012,1,1,강남구,개포동,649,0,120800,50000,3430,...,0.000000,25.496063,27.625902,2.265951,-3.234592,-9.974792,9.760000,21.407036,3.092783,56.621006
4862,2012,1,2,강남구,개포동,12,0,26500,23500,1172,...,1.471861,-24.901522,8.565093,-10.874525,-37.480831,12.623406,-12.012012,-35.267967,28.191141,-1.512605
4864,2012,1,2,강남구,개포동,138,0,60000,12000,665,...,-11.920529,-3.074164,46.092037,8.660131,-10.886678,48.957298,10.465117,-24.552027,26.275913,-7.638889


In [ ]:
final_df3.tail(10)

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,year_rent_price_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before
24144096,2022,12,31,중랑구,중화동,307,6,43900,30000,1234,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.603622,0.000000,0.000000,-22.344172,0.000000
24144097,2022,12,31,중랑구,중화동,307,76,37800,33600,1304,...,0.000000,0.000000,0.000000,0.000000,0.000000,11.258278,0.000000,0.000000,11.258278,0.000000
24144098,2022,12,31,중랑구,중화동,314,1,74000,37000,2244,...,54.332874,0.000000,0.000000,1.538462,0.000000,60.967545,1.081081,4.225352,-21.276596,26.923077
24144099,2022,12,31,중랑구,중화동,318,81,32500,12000,298,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24144100,2022,12,31,중랑구,중화동,331,64,76500,15500,1000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24144101,2022,12,31,중랑구,중화동,438,0,69500,32000,389,...,0.000000,5.677706,0.000000,0.000000,11.200000,17.216118,0.000000,11.200000,39.130436,0.000000
24144102,2022,12,31,중랑구,중화동,450,0,91500,32500,2090,...,2.955665,0.000000,-5.531494,-1.415094,25.342466,0.135568,17.021276,9.975962,1.217727,8.066184
24144103,2022,12,31,중랑구,중화동,452,0,55000,33000,1610,...,0.000000,0.000000,-26.666666,8.783784,0.000000,-26.666666,8.783784,0.000000,-26.666666,8.783784
24144104,2022,12,31,중랑구,중화동,453,0,85500,57750,2070,...,0.000000,0.000000,-0.431034,0.000000,0.000000,15.500000,0.000000,6.875000,15.500000,3.967855
24144105,2022,12,31,중랑구,중화동,454,0,96000,40000,2140,...,0.000000,0.000000,-4.761905,0.000000,0.000000,-4.761905,0.000000,0.608893,-11.406423,0.000000


In [ ]:
# 변화율 소수점 자리 수정
final_df3['deal_price_mean_1m_before'] = final_df3['deal_price_mean_1m_before'].round(2)
final_df3['full_rent_price_mean_1m_before'] = final_df3['full_rent_price_mean_1m_before'].round(2)
final_df3['year_rent_price_mean_1m_before'] = final_df3['year_rent_price_mean_1m_before'].round(2)

final_df3['deal_price_mean_3m_before'] = final_df3['deal_price_mean_3m_before'].round(2)
final_df3['full_rent_price_mean_3m_before'] = final_df3['full_rent_price_mean_3m_before'].round(2)
final_df3['year_rent_price_mean_3m_before'] = final_df3['year_rent_price_mean_3m_before'].round(2)

final_df3['deal_price_mean_6m_before'] = final_df3['deal_price_mean_6m_before'].round(2)
final_df3['full_rent_price_mean_6m_before'] = final_df3['full_rent_price_mean_6m_before'].round(2)
final_df3['year_rent_price_mean_6m_before'] = final_df3['year_rent_price_mean_6m_before'].round(2)

final_df3['deal_price_mean_12m_before'] = final_df3['deal_price_mean_12m_before'].round(2)
final_df3['full_rent_price_mean_12m_before'] = final_df3['full_rent_price_mean_12m_before'].round(2)
final_df3['year_rent_price_mean_12m_before'] = final_df3['year_rent_price_mean_12m_before'].round(2)
final_df3.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,year_rent_price_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before
4847,2012,1,1,강남구,개포동,12,0,26500,15000,1172,...,1.47,-24.900000,-30.700001,-10.870000,-37.480000,-28.110001,-12.01,-35.270000,-18.180000,-1.51
4849,2012,1,1,강남구,개포동,138,0,60000,12000,665,...,-11.92,-3.070000,46.090000,8.660000,-10.890000,48.959999,10.47,-24.549999,26.280001,-7.64
4850,2012,1,1,강남구,개포동,140,0,98800,25000,1170,...,-2.74,42.490002,45.070000,33.410000,67.309998,39.830002,6.07,9.740000,64.169998,-4.72
4851,2012,1,1,강남구,개포동,141,0,96500,11500,731,...,-12.24,23.070000,2.380000,-13.080000,15.410000,19.070000,-0.14,32.349998,9.810000,-9.08
4854,2012,1,1,강남구,개포동,185,0,80800,35500,1644,...,1.67,4.530000,5.190000,-19.610001,7.690000,10.960000,-8.31,-1.840000,31.700001,-1.26


In [ ]:
final_df3.head(10)

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,year_rent_price_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before
4847,2012,1,1,강남구,개포동,12,0,26500,15000,1172,...,1.470000,-24.900000,-30.700001,-10.870000,-37.480000,-28.110001,-12.010000,-35.270000,-18.180000,-1.510000
4849,2012,1,1,강남구,개포동,138,0,60000,12000,665,...,-11.920000,-3.070000,46.090000,8.660000,-10.890000,48.959999,10.470000,-24.549999,26.280001,-7.640000
4850,2012,1,1,강남구,개포동,140,0,98800,25000,1170,...,-2.740000,42.490002,45.070000,33.410000,67.309998,39.830002,6.070000,9.740000,64.169998,-4.720000
4851,2012,1,1,강남구,개포동,141,0,96500,11500,731,...,-12.240000,23.070000,2.380000,-13.080000,15.410000,19.070000,-0.140000,32.349998,9.810000,-9.080000
4854,2012,1,1,강남구,개포동,185,0,80800,35500,1644,...,1.670000,4.530000,5.190000,-19.610001,7.690000,10.960000,-8.310000,-1.840000,31.700001,-1.260000
4855,2012,1,1,강남구,개포동,187,0,61000,22000,2390,...,18.200001,-26.469999,-32.770000,38.470001,-19.150000,-32.220001,58.380001,31.340000,-26.100000,25.330000
4856,2012,1,1,강남구,개포동,189,0,63000,10000,680,...,-8.230000,-4.110000,4.040000,-2.440000,-7.920000,0.910000,2.260000,51.599998,11.210000,-5.420000
4857,2012,1,1,강남구,개포동,649,0,120800,50000,3430,...,0.000000,25.500000,27.629999,2.270000,-3.230000,-9.970000,9.760000,21.410000,3.090000,56.619999
4862,2012,1,2,강남구,개포동,12,0,26500,23500,1172,...,1.470000,-24.900000,8.570000,-10.870000,-37.480000,12.620000,-12.010000,-35.270000,28.190001,-1.510000
4864,2012,1,2,강남구,개포동,138,0,60000,12000,665,...,-11.920000,-3.070000,46.090000,8.660000,-10.890000,48.959999,10.470000,-24.549999,26.280001,-7.640000


In [ ]:
final_df3.tail(10)

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,year_rent_price_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before
24144096,2022,12,31,중랑구,중화동,307,6,43900,30000,1234,...,0.000000,0.00,0.00,0.00,0.00,0.600000,0.00,0.00,-22.340000,0.00
24144097,2022,12,31,중랑구,중화동,307,76,37800,33600,1304,...,0.000000,0.00,0.00,0.00,0.00,11.260000,0.00,0.00,11.260000,0.00
24144098,2022,12,31,중랑구,중화동,314,1,74000,37000,2244,...,54.330002,0.00,0.00,1.54,0.00,60.970001,1.08,4.23,-21.280001,26.92
24144099,2022,12,31,중랑구,중화동,318,81,32500,12000,298,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00
24144100,2022,12,31,중랑구,중화동,331,64,76500,15500,1000,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00
24144101,2022,12,31,중랑구,중화동,438,0,69500,32000,389,...,0.000000,5.68,0.00,0.00,11.20,17.219999,0.00,11.20,39.130001,0.00
24144102,2022,12,31,중랑구,중화동,450,0,91500,32500,2090,...,2.960000,0.00,-5.53,-1.42,25.34,0.140000,17.02,9.98,1.220000,8.07
24144103,2022,12,31,중랑구,중화동,452,0,55000,33000,1610,...,0.000000,0.00,-26.67,8.78,0.00,-26.670000,8.78,0.00,-26.670000,8.78
24144104,2022,12,31,중랑구,중화동,453,0,85500,57750,2070,...,0.000000,0.00,-0.43,0.00,0.00,15.500000,0.00,6.88,15.500000,3.97
24144105,2022,12,31,중랑구,중화동,454,0,96000,40000,2140,...,0.000000,0.00,-4.76,0.00,0.00,-4.760000,0.00,0.61,-11.410000,0.00


In [ ]:
final_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21523831 entries, 4847 to 24144105
Data columns (total 22 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   year                             int16  
 1   month                            int16  
 2   day                              int16  
 3   address_1                        object 
 4   address_2                        object 
 5   address_3                        int16  
 6   address_4                        int16  
 7   deal_price                       int32  
 8   full_rent_price                  int32  
 9   year_rent_price                  int32  
 10  deal_price_mean_1m_before        float32
 11  full_rent_price_mean_1m_before   float32
 12  year_rent_price_mean_1m_before   float32
 13  deal_price_mean_3m_before        float32
 14  full_rent_price_mean_3m_before   float32
 15  year_rent_price_mean_3m_before   float32
 16  deal_price_mean_6m_before        float32
 17  ful

In [ ]:
final_df3.to_pickle('/content/drive/MyDrive/house_price/final_df3.pkl')

# 아파트 거래 수정2(미래가격 컬럼추가)

## 미래가격 데이터프레임 생성

In [ ]:
# final_df2가 거래가격이0인 값들을 다 제거한 것이므로 불러옴옴
future_df = pd.read_pickle('/content/drive/MyDrive/house_price/final_df2.pkl')
future_df

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price
0,2011,1,5,강남구,개포동,138,0,89400,11000,778
1,2011,1,5,강남구,개포동,185,0,89000,22000,1786
2,2011,1,6,강남구,개포동,138,0,89400,7500,778
3,2011,1,6,강남구,개포동,185,0,89000,30666,1424
4,2011,1,7,강남구,개포동,12,0,35000,24000,1196
...,...,...,...,...,...,...,...,...,...,...
24144101,2022,12,31,중랑구,중화동,438,0,69500,32000,389
24144102,2022,12,31,중랑구,중화동,450,0,91500,32500,2090
24144103,2022,12,31,중랑구,중화동,452,0,55000,33000,1610
24144104,2022,12,31,중랑구,중화동,453,0,85500,57750,2070


In [ ]:
future_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24144106 entries, 0 to 24144105
Data columns (total 10 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   year             int16 
 1   month            int16 
 2   day              int16 
 3   address_1        object
 4   address_2        object
 5   address_3        int16 
 6   address_4        int16 
 7   deal_price       int32 
 8   full_rent_price  int32 
 9   year_rent_price  int32 
dtypes: int16(5), int32(3), object(2)
memory usage: 875.0+ MB


In [ ]:
# 날짜 컬럼 생성성
future_df = future_df.astype({'year':'str','month':'str','day':'str'})
future_df.loc[future_df["month"].str.len()==1,"month"]='0'+future_df.loc[future_df["month"].str.len()==1,"month"]
future_df.loc[future_df["day"].str.len()==1,"day"]='0'+future_df.loc[future_df["day"].str.len()==1,"day"] # 일이 있는 컬럼에서 1자리 숫자인 경우 앞에 0을 추가성
future_df['date'] = pd.to_datetime(future_df['year']+future_df['month']+future_df['day']) 
future_df = future_df.astype({'year':'int16','month':'int16','day':'int16'})
future_df.head(10)

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,date
0,2011,1,5,강남구,개포동,138,0,89400,11000,778,2011-01-05
1,2011,1,5,강남구,개포동,185,0,89000,22000,1786,2011-01-05
2,2011,1,6,강남구,개포동,138,0,89400,7500,778,2011-01-06
3,2011,1,6,강남구,개포동,185,0,89000,30666,1424,2011-01-06
4,2011,1,7,강남구,개포동,12,0,35000,24000,1196,2011-01-07
5,2011,1,7,강남구,개포동,138,0,89400,7000,778,2011-01-07
6,2011,1,7,강남구,개포동,141,0,80300,12000,1134,2011-01-07
7,2011,1,7,강남구,개포동,185,0,79000,24000,1424,2011-01-07
8,2011,1,8,강남구,개포동,12,0,35000,24000,1196,2011-01-08
9,2011,1,8,강남구,개포동,138,0,89400,9000,658,2011-01-08


In [ ]:
future_df.tail()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,date
24144101,2022,12,31,중랑구,중화동,438,0,69500,32000,389,2022-12-31
24144102,2022,12,31,중랑구,중화동,450,0,91500,32500,2090,2022-12-31
24144103,2022,12,31,중랑구,중화동,452,0,55000,33000,1610,2022-12-31
24144104,2022,12,31,중랑구,중화동,453,0,85500,57750,2070,2022-12-31
24144105,2022,12,31,중랑구,중화동,454,0,96000,40000,2140,2022-12-31


In [ ]:
# 1년 전 날짜 구하기(데이터 프레임과 합치면 미래가격을 구할 수 있음음)
future_df['date'] = future_df['date'] - pd.Timedelta(days=365)

In [ ]:
future_df

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,date
0,2011,1,5,강남구,개포동,138,0,89400,11000,778,2010-01-05
1,2011,1,5,강남구,개포동,185,0,89000,22000,1786,2010-01-05
2,2011,1,6,강남구,개포동,138,0,89400,7500,778,2010-01-06
3,2011,1,6,강남구,개포동,185,0,89000,30666,1424,2010-01-06
4,2011,1,7,강남구,개포동,12,0,35000,24000,1196,2010-01-07
...,...,...,...,...,...,...,...,...,...,...,...
24144101,2022,12,31,중랑구,중화동,438,0,69500,32000,389,2021-12-31
24144102,2022,12,31,중랑구,중화동,450,0,91500,32500,2090,2021-12-31
24144103,2022,12,31,중랑구,중화동,452,0,55000,33000,1610,2021-12-31
24144104,2022,12,31,중랑구,중화동,453,0,85500,57750,2070,2021-12-31


In [ ]:
future_df['year'] = future_df['date'].dt.year
future_df['month'] = future_df['date'].dt.month
future_df['day'] = future_df['date'].dt.day
future_df

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,date
0,2010,1,5,강남구,개포동,138,0,89400,11000,778,2010-01-05
1,2010,1,5,강남구,개포동,185,0,89000,22000,1786,2010-01-05
2,2010,1,6,강남구,개포동,138,0,89400,7500,778,2010-01-06
3,2010,1,6,강남구,개포동,185,0,89000,30666,1424,2010-01-06
4,2010,1,7,강남구,개포동,12,0,35000,24000,1196,2010-01-07
...,...,...,...,...,...,...,...,...,...,...,...
24144101,2021,12,31,중랑구,중화동,438,0,69500,32000,389,2021-12-31
24144102,2021,12,31,중랑구,중화동,450,0,91500,32500,2090,2021-12-31
24144103,2021,12,31,중랑구,중화동,452,0,55000,33000,1610,2021-12-31
24144104,2021,12,31,중랑구,중화동,453,0,85500,57750,2070,2021-12-31


In [ ]:
future_df.drop(["date","full_rent_price","year_rent_price"], axis=1,inplace=True)
future_df.columns = ['year','month','day','address_1','address_2','address_3','address_4','future_deal_price']
future_df

,year,month,day,address_1,address_2,address_3,address_4,future_deal_price
0,2010,1,5,강남구,개포동,138,0,89400
1,2010,1,5,강남구,개포동,185,0,89000
2,2010,1,6,강남구,개포동,138,0,89400
3,2010,1,6,강남구,개포동,185,0,89000
4,2010,1,7,강남구,개포동,12,0,35000
...,...,...,...,...,...,...,...,...
24144101,2021,12,31,중랑구,중화동,438,0,69500
24144102,2021,12,31,중랑구,중화동,450,0,91500
24144103,2021,12,31,중랑구,중화동,452,0,55000
24144104,2021,12,31,중랑구,중화동,453,0,85500


In [ ]:
future_df = future_df.astype({'year':'int16','month':'int16','day':'int16'})
future_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24144106 entries, 0 to 24144105
Data columns (total 8 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   year               int16 
 1   month              int16 
 2   day                int16 
 3   address_1          object
 4   address_2          object
 5   address_3          int16 
 6   address_4          int16 
 7   future_deal_price  int32 
dtypes: int16(5), int32(1), object(2)
memory usage: 690.8+ MB


In [ ]:
future_df.to_pickle('/content/drive/MyDrive/house_price/future_df.pkl')

## 미래가격과 final_df3 merge

In [ ]:
# 파일들 불러오기
future_df = pd.read_pickle('/content/drive/MyDrive/house_price/future_df.pkl')
final_df3 = pd.read_pickle('/content/drive/MyDrive/house_price/final_df3.pkl')

In [ ]:
final_df4 = pd.merge(final_df3, future_df, on=['address_1','address_2','address_3','address_4',
                                   'year','month','day'], how = 'inner')

In [ ]:
final_df4.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before,future_deal_price
0,2012,1,1,강남구,개포동,12,0,26500,15000,1172,...,-24.900000,-30.700001,-10.870000,-37.480000,-28.110001,-12.01,-35.270000,-18.180000,-1.51,40000
1,2012,1,1,강남구,개포동,138,0,60000,12000,665,...,-3.070000,46.090000,8.660000,-10.890000,48.959999,10.47,-24.549999,26.280001,-7.64,45000
2,2012,1,1,강남구,개포동,140,0,98800,25000,1170,...,42.490002,45.070000,33.410000,67.309998,39.830002,6.07,9.740000,64.169998,-4.72,85750
3,2012,1,1,강남구,개포동,141,0,96500,11500,731,...,23.070000,2.380000,-13.080000,15.410000,19.070000,-0.14,32.349998,9.810000,-9.08,86500
4,2012,1,1,강남구,개포동,185,0,80800,35500,1644,...,4.530000,5.190000,-19.610001,7.690000,10.960000,-8.31,-1.840000,31.700001,-1.26,49000


In [ ]:
final_df4.tail()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before,future_deal_price
18875827,2021,12,31,중랑구,중화동,438,0,62500,23000,389,...,9.650000,0.000000,0.00,23.969999,-23.33,0.00,68.919998,-23.330000,0.000000,69500
18875828,2021,12,31,중랑구,중화동,450,0,83200,31500,1336,...,17.379999,-26.340000,-20.24,16.530001,-24.85,-13.86,37.320000,-9.900000,-24.090000,91500
18875829,2021,12,31,중랑구,중화동,452,0,55000,45000,1480,...,0.000000,0.000000,0.00,0.000000,8.87,0.00,0.000000,36.360001,0.000000,55000
18875830,2021,12,31,중랑구,중화동,453,0,80000,50000,2070,...,0.000000,0.000000,13.11,0.000000,-9.09,-13.53,19.940001,-9.090000,7.250000,85500
18875831,2021,12,31,중랑구,중화동,454,0,96000,45150,2140,...,6.670000,-17.309999,0.00,7.870000,-17.91,0.00,24.100000,8.110000,16.940001,96000


In [ ]:
final_df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18875832 entries, 0 to 18875831
Data columns (total 23 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   year                             int16  
 1   month                            int16  
 2   day                              int16  
 3   address_1                        object 
 4   address_2                        object 
 5   address_3                        int16  
 6   address_4                        int16  
 7   deal_price                       int32  
 8   full_rent_price                  int32  
 9   year_rent_price                  int32  
 10  deal_price_mean_1m_before        float32
 11  full_rent_price_mean_1m_before   float32
 12  year_rent_price_mean_1m_before   float32
 13  deal_price_mean_3m_before        float32
 14  full_rent_price_mean_3m_before   float32
 15  year_rent_price_mean_3m_before   float32
 16  deal_price_mean_6m_before        float32
 17  full_r

In [ ]:
final_df4.isnull().sum()

year                               0
month                              0
day                                0
address_1                          0
address_2                          0
address_3                          0
address_4                          0
deal_price                         0
full_rent_price                    0
year_rent_price                    0
deal_price_mean_1m_before          0
full_rent_price_mean_1m_before     0
year_rent_price_mean_1m_before     0
deal_price_mean_3m_before          0
full_rent_price_mean_3m_before     0
year_rent_price_mean_3m_before     0
deal_price_mean_6m_before          0
full_rent_price_mean_6m_before     0
year_rent_price_mean_6m_before     0
deal_price_mean_12m_before         0
full_rent_price_mean_12m_before    0
year_rent_price_mean_12m_before    0
future_deal_price                  0
dtype: int64

In [ ]:
final_df4.to_pickle('/content/drive/MyDrive/house_price/final_df4.pkl')

# 아파트 거래가격 + 거래량 생성

In [ ]:
final_df4 = pd.read_pickle('/content/drive/MyDrive/house_price/final_df4.pkl')
final_df4.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before,future_deal_price
0,2012,1,1,강남구,개포동,12,0,26500,15000,1172,...,-24.900000,-30.700001,-10.870000,-37.480000,-28.110001,-12.01,-35.270000,-18.180000,-1.51,40000
1,2012,1,1,강남구,개포동,138,0,60000,12000,665,...,-3.070000,46.090000,8.660000,-10.890000,48.959999,10.47,-24.549999,26.280001,-7.64,45000
2,2012,1,1,강남구,개포동,140,0,98800,25000,1170,...,42.490002,45.070000,33.410000,67.309998,39.830002,6.07,9.740000,64.169998,-4.72,85750
3,2012,1,1,강남구,개포동,141,0,96500,11500,731,...,23.070000,2.380000,-13.080000,15.410000,19.070000,-0.14,32.349998,9.810000,-9.08,86500
4,2012,1,1,강남구,개포동,185,0,80800,35500,1644,...,4.530000,5.190000,-19.610001,7.690000,10.960000,-8.31,-1.840000,31.700001,-1.26,49000


In [ ]:
final_df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18875832 entries, 0 to 18875831
Data columns (total 23 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   year                             int16  
 1   month                            int16  
 2   day                              int16  
 3   address_1                        object 
 4   address_2                        object 
 5   address_3                        int16  
 6   address_4                        int16  
 7   deal_price                       int32  
 8   full_rent_price                  int32  
 9   year_rent_price                  int32  
 10  deal_price_mean_1m_before        float32
 11  full_rent_price_mean_1m_before   float32
 12  year_rent_price_mean_1m_before   float32
 13  deal_price_mean_3m_before        float32
 14  full_rent_price_mean_3m_before   float32
 15  year_rent_price_mean_3m_before   float32
 16  deal_price_mean_6m_before        float32
 17  full_r

In [ ]:
# 아파트 거래량량 테이블 불러오기기
final_count = pd.read_pickle('/content/drive/MyDrive/house_price/month_region_count.pkl')
final_count.head()

,address_1,address_2,address_3,year,month,last_month_deal_count,last_month_full_rent_count,last_month_month_rent_count,deal_count_1m_before,full_rent_count_1m_before,month_rent_count_1m_before,deal_count_3m_before,full_rent_count_3m_before,month_rent_count_3m_before,deal_count_6m_before,full_rent_count_6m_before,month_rent_count_6m_before,deal_count_12m_before,full_rent_count_12m_before,month_rent_count_12m_before
13,강남구,개포동,12,2012,2,7,33,9,-2,1,0,-1,-7,-5,3,0,-4,-9,-5,2
14,강남구,개포동,12,2012,3,7,42,9,0,9,0,3,6,4,-1,13,0,-10,4,1
15,강남구,개포동,12,2012,4,5,49,9,-2,7,0,-4,17,0,-3,21,-4,-3,3,-7
16,강남구,개포동,12,2012,5,1,35,12,-4,-14,3,-6,2,3,-7,-5,-2,-12,5,1
17,강남구,개포동,12,2012,6,6,26,8,5,-9,-4,-1,-16,-1,2,-10,3,-1,5,0


In [ ]:
final_count.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 904162 entries, 13 to 993887
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   address_1                    904162 non-null  object
 1   address_2                    904162 non-null  object
 2   address_3                    904162 non-null  int16 
 3   year                         904162 non-null  int16 
 4   month                        904162 non-null  int16 
 5   last_month_deal_count        904162 non-null  int16 
 6   last_month_full_rent_count   904162 non-null  int16 
 7   last_month_month_rent_count  904162 non-null  int16 
 8   deal_count_1m_before         904162 non-null  int16 
 9   full_rent_count_1m_before    904162 non-null  int16 
 10  month_rent_count_1m_before   904162 non-null  int16 
 11  deal_count_3m_before         904162 non-null  int16 
 12  full_rent_count_3m_before    904162 non-null  int16 
 13  month_rent_co

In [ ]:
final_df5 = pd.merge(final_df4, final_count, on=['year','month','address_1','address_2','address_3'])
final_df5

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,month_rent_count_1m_before,deal_count_3m_before,full_rent_count_3m_before,month_rent_count_3m_before,deal_count_6m_before,full_rent_count_6m_before,month_rent_count_6m_before,deal_count_12m_before,full_rent_count_12m_before,month_rent_count_12m_before
0,2012,2,1,강남구,개포동,12,0,50500,19000,1210,...,0,-1,-7,-5,3,0,-4,-9,-5,2
1,2012,2,2,강남구,개포동,12,0,50100,19000,1210,...,0,-1,-7,-5,3,0,-4,-9,-5,2
2,2012,2,3,강남구,개포동,12,0,50100,30000,1210,...,0,-1,-7,-5,3,0,-4,-9,-5,2
3,2012,2,4,강남구,개포동,12,0,50100,24500,1194,...,0,-1,-7,-5,3,0,-4,-9,-5,2
4,2012,2,5,강남구,개포동,12,0,50100,23000,1194,...,0,-1,-7,-5,3,0,-4,-9,-5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18853197,2021,12,27,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0
18853198,2021,12,28,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0
18853199,2021,12,29,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0
18853200,2021,12,30,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0


In [ ]:
final_df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18853202 entries, 0 to 18853201
Data columns (total 38 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   year                             int16  
 1   month                            int16  
 2   day                              int16  
 3   address_1                        object 
 4   address_2                        object 
 5   address_3                        int16  
 6   address_4                        int16  
 7   deal_price                       int32  
 8   full_rent_price                  int32  
 9   year_rent_price                  int32  
 10  deal_price_mean_1m_before        float32
 11  full_rent_price_mean_1m_before   float32
 12  year_rent_price_mean_1m_before   float32
 13  deal_price_mean_3m_before        float32
 14  full_rent_price_mean_3m_before   float32
 15  year_rent_price_mean_3m_before   float32
 16  deal_price_mean_6m_before        float32
 17  full_r

In [ ]:
final_df5.isnull().sum()

year                               0
month                              0
day                                0
address_1                          0
address_2                          0
address_3                          0
address_4                          0
deal_price                         0
full_rent_price                    0
year_rent_price                    0
deal_price_mean_1m_before          0
full_rent_price_mean_1m_before     0
year_rent_price_mean_1m_before     0
deal_price_mean_3m_before          0
full_rent_price_mean_3m_before     0
year_rent_price_mean_3m_before     0
deal_price_mean_6m_before          0
full_rent_price_mean_6m_before     0
year_rent_price_mean_6m_before     0
deal_price_mean_12m_before         0
full_rent_price_mean_12m_before    0
year_rent_price_mean_12m_before    0
future_deal_price                  0
last_month_deal_count              0
last_month_full_rent_count         0
last_month_month_rent_count        0
deal_count_1m_before               0
f

In [ ]:
final_df5.to_pickle('/content/drive/MyDrive/house_price/final_df5.pkl')

# 아파트 거래 + 거래량 + 경제지표 생성

## 용량이 크기에 slice 해서 실행행

In [3]:
final_df5 = pd.read_pickle('/content/drive/MyDrive/house_price/final_df5.pkl')
final_df5.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,month_rent_count_1m_before,deal_count_3m_before,full_rent_count_3m_before,month_rent_count_3m_before,deal_count_6m_before,full_rent_count_6m_before,month_rent_count_6m_before,deal_count_12m_before,full_rent_count_12m_before,month_rent_count_12m_before
0,2012,2,1,강남구,개포동,12,0,50500,19000,1210,...,0,-1,-7,-5,3,0,-4,-9,-5,2
1,2012,2,2,강남구,개포동,12,0,50100,19000,1210,...,0,-1,-7,-5,3,0,-4,-9,-5,2
2,2012,2,3,강남구,개포동,12,0,50100,30000,1210,...,0,-1,-7,-5,3,0,-4,-9,-5,2
3,2012,2,4,강남구,개포동,12,0,50100,24500,1194,...,0,-1,-7,-5,3,0,-4,-9,-5,2
4,2012,2,5,강남구,개포동,12,0,50100,23000,1194,...,0,-1,-7,-5,3,0,-4,-9,-5,2


In [4]:
# 용량이 크기에 나눠서 실행행
real_final_df_list = [final_df5.iloc[i:i+100000] for i in range(0, len(final_df5 ), 100000)]

In [5]:
len(real_final_df_list)

189

In [6]:
real_final_df_list[188]

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,month_rent_count_1m_before,deal_count_3m_before,full_rent_count_3m_before,month_rent_count_3m_before,deal_count_6m_before,full_rent_count_6m_before,month_rent_count_6m_before,deal_count_12m_before,full_rent_count_12m_before,month_rent_count_12m_before
18800000,2021,1,29,중랑구,중화동,42,0,42500,24000,605,...,0,0,3,0,-2,1,0,-1,2,0
18800001,2021,1,30,중랑구,중화동,42,0,42500,24000,605,...,0,0,3,0,-2,1,0,-1,2,0
18800002,2021,1,31,중랑구,중화동,42,0,42500,24000,605,...,0,0,3,0,-2,1,0,-1,2,0
18800003,2021,1,1,중랑구,중화동,110,26,40000,25500,1300,...,0,1,0,0,0,0,0,1,-1,0
18800004,2021,1,2,중랑구,중화동,110,26,40000,25500,1300,...,0,1,0,0,0,0,0,1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18853197,2021,12,27,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0
18853198,2021,12,28,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0
18853199,2021,12,29,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0
18853200,2021,12,30,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0


In [7]:
final_economic2 = pd.read_pickle('/content/drive/MyDrive/house_price/final_economic2.pkl')
final_economic2.head()

,date,year,month,day,apartment_index,kospi_index,korea_rp,korea_3_year,korea_10_year,korea_10-3_year,...,us_2_year_12m_before_change_rate,us_10_year_12m_before_change_rate,us_10-2_year_12m_before_change_rate,us_10-3_year_month_12m_before_change_rate,apartment_supply_12m_before_change_rate,unsold_count_12m_before_change_rate,unsold_ratio_12m_before_change_rate,last_month_deal_count_12m_before_change_rate,last_month_full_rent_count_12m_before_change_rate,last_month_month_rent_count_12m_before_change_rate
0,2012-01-01,2012,1,1,86.8,1825.74,3.25,3.32,3.77,0.45,...,-58.901830,-43.731254,-40.395170,-42.180685,30.215154,-17.98149,-37.013084,-48.293634,-20.06323,-20.922832
1,2012-01-02,2012,1,2,86.8,1826.37,3.25,3.34,3.78,0.44,...,-58.901830,-43.821236,-40.504940,-42.274143,30.215154,-17.98149,-37.013084,-48.293634,-20.06323,-20.922832
2,2012-01-03,2012,1,3,86.8,1875.41,3.25,3.36,3.79,0.43,...,-56.905158,-41.331734,-37.907062,-39.688474,30.215154,-17.98149,-37.013084,-48.293634,-20.06323,-20.922832
3,2012-01-04,2012,1,4,86.8,1866.22,3.25,3.36,3.79,0.43,...,-57.648953,-40.563212,-36.658079,-38.391740,30.215154,-17.98149,-37.013084,-48.293634,-20.06323,-20.922832
4,2012-01-05,2012,1,5,86.8,1863.74,3.25,3.34,3.78,0.44,...,-62.853107,-42.362114,-37.096189,-40.349292,30.215154,-17.98149,-37.013084,-48.293634,-20.06323,-20.922832


In [8]:
final_economic2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4018 entries, 0 to 4017
Data columns (total 85 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                4018 non-null   object 
 1   year                                                4018 non-null   int16  
 2   month                                               4018 non-null   int16  
 3   day                                                 4018 non-null   int16  
 4   apartment_index                                     4018 non-null   float64
 5   kospi_index                                         4018 non-null   float64
 6   korea_rp                                            4018 non-null   float64
 7   korea_3_year                                        4018 non-null   float64
 8   korea_10_year                                       4018 non-null   float64
 9

### 범위를 한번에 하면 메모리 오류로 인해서 나눠서 실행(0-30,30-60,60-90,90-120,120-150,150-189)

In [9]:
temp_list = list()
num_list = [[0,30],[30,60],[60,90],[60,90],[90,120],[120,150],[150,189]]
# real_final_df1 에 사용
# for i in range(num_list[0][0],num_list[0][1]):
#   temp_list.append(pd.merge(real_final_df_list[i], final_economic2, on=['year','month','day']))
#   print(i)

# real_final_df2 에 사용
for i in range(num_list[1][0],num_list[1][1]):
  temp_list.append(pd.merge(real_final_df_list[i], final_economic2, on=['year','month','day']))
  print(i)

30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [10]:
for i,temp_df in enumerate(temp_list):
  print(i)
  if i == 0:
    real_final_df=temp_df.copy()
  else:
    real_final_df = pd.concat([real_final_df, temp_df], axis=0)
real_final_df.reset_index(drop=True,inplace=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [11]:
real_final_df

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,us_2_year_12m_before_change_rate,us_10_year_12m_before_change_rate,us_10-2_year_12m_before_change_rate,us_10-3_year_month_12m_before_change_rate,apartment_supply_12m_before_change_rate,unsold_count_12m_before_change_rate,unsold_ratio_12m_before_change_rate,last_month_deal_count_12m_before_change_rate,last_month_full_rent_count_12m_before_change_rate,last_month_month_rent_count_12m_before_change_rate
0,2019,6,21,강북구,수유동,50,20,24300,16000,1076,...,-30.270355,-28.975509,-19.782062,-105.279503,-28.650138,278.723404,430.797667,-6.381174,9.479058,3.194245
1,2019,6,21,강북구,수유동,50,75,34000,22000,1043,...,-30.270355,-28.975509,-19.782062,-105.279503,-28.650138,278.723404,430.797667,-6.381174,9.479058,3.194245
2,2019,6,21,강북구,수유동,51,7,35000,20500,1814,...,-30.270355,-28.975509,-19.782062,-105.279503,-28.650138,278.723404,430.797667,-6.381174,9.479058,3.194245
3,2019,6,21,강북구,수유동,63,0,30500,20000,1192,...,-30.270355,-28.975509,-19.782062,-105.279503,-28.650138,278.723404,430.797667,-6.381174,9.479058,3.194245
4,2019,6,21,강북구,수유동,77,9,20200,13000,652,...,-30.270355,-28.975509,-19.782062,-105.279503,-28.650138,278.723404,430.797667,-6.381174,9.479058,3.194245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2012,12,29,구로구,구로동,743,27,6700,4500,292,...,-7.601476,-10.426540,-10.896806,-10.487030,-73.618491,99.555802,656.423001,23.586246,22.776968,36.368623
99996,2012,12,30,구로구,구로동,743,10,8900,7200,418,...,1.376518,-9.328358,-10.951504,-8.895474,-73.618491,99.555802,656.423001,23.586246,22.776968,36.368623
99997,2012,12,30,구로구,구로동,743,27,6700,4500,292,...,1.376518,-9.328358,-10.951504,-8.895474,-73.618491,99.555802,656.423001,23.586246,22.776968,36.368623
99998,2012,12,31,구로구,구로동,743,10,8900,7200,418,...,1.619433,-6.396588,-7.612032,-7.866379,-73.618491,99.555802,656.423001,23.586246,22.776968,36.368623


In [12]:
real_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000000 entries, 0 to 99999
Columns: 120 entries, year to last_month_month_rent_count_12m_before_change_rate
dtypes: float32(12), float64(76), int16(20), int32(9), object(3)
memory usage: 2.1+ GB


In [13]:
# 전세가율, 연세가율? 구함함
real_final_df['deal_to_full_rent_rate'] = real_final_df['deal_price']/real_final_df['full_rent_price']
real_final_df['deal_to_year_rent_rate'] = real_final_df['deal_price']/real_final_df['year_rent_price']

In [14]:
# 타입변경경
real_final_df = real_final_df.astype({'deal_to_full_rent_rate':'float32', 'deal_to_year_rent_rate':'float32'})
real_final_df['deal_to_full_rent_rate'] = real_final_df['deal_to_full_rent_rate'].round(2)
real_final_df['deal_to_year_rent_rate'] = real_final_df['deal_to_year_rent_rate'].round(2)

In [18]:
# real_final_df1 에 사용
# real_final_df.to_pickle('/content/drive/MyDrive/house_price/real_final_df1.pkl')

# real_final_df2 에 사용
# real_final_df.to_pickle('/content/drive/MyDrive/house_price/real_final_df2.pkl')